# A Gentle Introduction to RAG Applications

This notebook creates a simple RAG (Retrieval-Augmented Generation) system to answer questions from a PDF document using an open-source model.

In [40]:
PDF_FILE = "basedata.pdf"

# We'll be using Llama 3.1 8B for this example.
MODEL = "llama3.1"

## Loading the PDF document

Let's start by loading the PDF document and breaking it down into separate pages.

<img src='images/documents.png' width="1000">

In [41]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(PDF_FILE)
pages = loader.load()

print(f"Number of pages: {len(pages)}")
print(f"Length of a page: {len(pages[1].page_content)}")
print("Content of a page:", pages[1].page_content)

Number of pages: 6
Length of a page: 1579
Content of a page: 2. Payer les frais via les moyens de paiement mobile (©Flooz, ©Tmoney) et par carte bancaire 
(©Visa, ©Mastercard) disponibles sur la plateforme ; 
3. Les informations fournies à travers le formulaire sont transmises au centre de traitement que 
vous avez choisi. Votre centre procède à une vérification de vos données : 
o En cas d'erreur ou de non-conformité d'une information fournie, une notification 
contenant les consignes de correction est envoyée sur votre compte utilisateur et par 
SMS et par e-mail. Une fois l'erreur corrigée, les informations sont renvoyées au centre 
de traitement ; 
o En cas de conformité des informations fournies, un message est envoyé sur votre 
compte utilisateur, par SMS et par e-mail à chaque étape du traitement jusqu'à la mise à 
disposition de votre duplicata ; 
4. Le retrait de votre duplicata se fait suivant le mode de retrait que vous avez choisi (dans un 
centre de traitement ou via la po

## Splitting the pages in chunks

Pages are too long, so let's split pages into different chunks.

<img src='images/splitter.png' width="1000">


In [59]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)

chunks = splitter.split_documents(pages)
print(f"Number of chunks: {len(chunks)}")
print(f"Length of a chunk: {len(chunks[1].page_content)}")
print("Content of a chunk:", chunks[1].page_content)


Number of chunks: 10
Length of a chunk: 1498
Content of a chunk: 2. Payer les frais via les moyens de paiement mobile (©Flooz, ©Tmoney) et par carte bancaire 
(©Visa, ©Mastercard) disponibles sur la plateforme ; 
3. Les informations fournies à travers le formulaire sont transmises au centre de traitement que 
vous avez choisi. Votre centre procède à une vérification de vos données : 
o En cas d'erreur ou de non-conformité d'une information fournie, une notification 
contenant les consignes de correction est envoyée sur votre compte utilisateur et par 
SMS et par e-mail. Une fois l'erreur corrigée, les informations sont renvoyées au centre 
de traitement ; 
o En cas de conformité des informations fournies, un message est envoyé sur votre 
compte utilisateur, par SMS et par e-mail à chaque étape du traitement jusqu'à la mise à 
disposition de votre duplicata ; 
4. Le retrait de votre duplicata se fait suivant le mode de retrait que vous avez choisi (dans un 
centre de traitement ou via l

## Storing the chunks in a vector store

We can now generate embeddings for every chunk and store them in a vector store.

<img src='images/vectorstore.png' width="1000">


In [60]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model=MODEL)
vectorstore = FAISS.from_documents(chunks, embeddings)

## Setting up a retriever

We can use a retriever to find chunks in the vector store that are similar to a supplied question.

<img src='images/retriever.png' width="1000">



In [64]:
retriever = vectorstore.as_retriever()
retriever.invoke("Electrique ")

[Document(metadata={'source': 'basedata.pdf', 'page': 3}, page_content='Quant au passeport diplomatique, la demande est faite par le Ministère des Affaires Etrangères et \nadressée au Ministère de la Sécurité qui la transmet à la Direction Générale de la Documentation \nNationale.'),
 Document(metadata={'source': 'basedata.pdf', 'page': 4}, page_content='documents sus-mentionnés, une copie de la lettre que son ministère de tutelle a envoyé au Ministère de \nla Sécurité. \n \nÉtapes \nAttention : la présente procédure dématérialisée porte sur la demande de passeport ordinaire et la \ndemande de passeport de service. La demande en ligne du passeport diplomatique pour les diplomates \net hauts fonctionnaires de l’Etat sera bientôt intégrée à la présente plateforme.'),
 Document(metadata={'source': 'basedata.pdf', 'page': 1}, page_content="1 180 F CFA pour les frais de traitement des dossiers \n \nDurée de validité \nÀ vie \nDescription \nCet abonnement permet l'accès à l'électricité des m

## Configuring the model

We'll be using Ollama to load the local model in memory. After creating the model, we can invoke it with a question to get the response back.

<img src='images/model.png' width="1000">

In [66]:
from langchain_ollama import ChatOllama

model = ChatOllama(model=MODEL, temperature=0)
model.invoke("tu connais la ceet au togo ?")

AIMessage(content="La CEET (Compagnie d'Électricité du Togo) est une entreprise publique togolaise chargée de la production, transmission et distribution d'électricité dans le pays. Elle a pour mission de fournir un service public d'électricité fiable, efficace et accessible à tous les citoyens du Togo.\n\nLa CEET a été créée en 1961 et est depuis lors responsable de l'approvisionnement électrique du pays. Elle dispose d'une capacité de production électrique qui répond aux besoins énergétiques du Togo, notamment grâce à des centrales thermiques et hydroélectriques.\n\nLa CEET a également mis en place un réseau de distribution électrique qui couvre l'ensemble du territoire national, y compris les zones rurales. Elle propose diverses options de tarifs pour les consommateurs, notamment des offres spéciales pour les ménages à faible revenu.\n\nEn outre, la CEET travaille également à améliorer la qualité et la fiabilité du service électrique qu'elle fournit, en mettant en place des systèmes

## Parsing the model's response

The response from the model is an `AIMessage` instance containing the answer. We can extract the text answer by using the appropriate output parser. We can connect the model and the parser using a chain.

<img src='images/parser.png' width="1000">


In [67]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
print(chain.invoke("tu connais la ceet au togo ?"))

La CEET (Compagnie d'Électricité du Togo) est une entreprise publique togolaise chargée de la production, transmission et distribution d'électricité dans le pays. Elle a pour mission de fournir un service public d'électricité fiable, efficace et accessible à tous les citoyens du Togo.

La CEET a été créée en 1961 et est depuis lors responsable de l'approvisionnement électrique du pays. Elle dispose d'une capacité de production électrique qui permet de répondre aux besoins énergétiques des ménages, des entreprises et des institutions publiques.

Malheureusement, je n'ai pas d'informations plus précises sur la CEET au Togo, comme son siège social, ses activités, ses chiffres clés, etc. Si vous avez besoin de plus d'informations ou que vous souhaitez en savoir plus sur ce sujet, n'hésitez pas à me le faire savoir !


## Setting up a prompt

In addition to the question we want to ask, we also want to provide the model with the context from the PDF file. We can use a prompt template to define and reuse the prompt we'll use with the model.


<img src='images/prompt.png' width="1000">

In [ ]:
from langchain.prompts import PromptTemplate

template = """
You are TogoBot, an assistant that provides answers to questions based on a given context. Introduce yourself in less than 5 words.

Answer the question in French based on the context. If you don't know the answer, say something cool like "Uhmm.. Interesting ! I don't know I will search".

Be as concise as possible and go straight to the point in French.

Context: {context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


You are TogoBot, an assistant that provides answers to questions based on a given context. Introduce yourself in less than 5 words.

Answer the question in French based on the context. If you don't know the answer, say something cool like "Uhmm.. Interesting ! I don't know I will search".

Be as concise as possible and go straight to the point in French.

Context: Here is some context

Question: Here is a question



## Adding the prompt to the chain

We can now chain the prompt with the model and the parser.

<img src='images/chain1.png' width="1000">

In [69]:
chain = prompt | model | parser

chain.invoke({
    "context": "La nationalite est le document qui prouve ta nationalité", 
    "question": "donne moi le document qui prouve ta nationalité"
})


'Je suis TogoBot, votre assistant !\n\nLa réponse est : La nationalité.'

## Adding the retriever to the chain

Finally, we can connect the retriever to the chain to get the context from the vector store.

<img src='images/chain2.png' width="1000">

In [70]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

## Using the chain to answer questions

Finally, we can use the chain to ask questions that will be answered using the PDF document.

In [ ]:
questions = "tu connais la Compagnie Energie Electrique du Togo  ? ",


for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print("*************************\n")

Question: tu connais la Compagnie Energie Electrique du Togo  ? 
Answer: Oui, je connais la CEE. Elle est responsable de la fourniture d'électricité dans le pays.
*************************

